In [1]:
import geopandas as gpd

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import src

/Users/lcuartase/.conda/envs/ox_eafit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Zonas Medellin

In [2]:
# List of areas of interest
zo = ['floresta_HH','moravia_LH','aguacatala_HL']
# zo = ['moravia_LH']
# z = zo[0]
for z in zo:
    # Show the zone that is running
    print(f"Procesando zona: {z}")

    # Load nodes and edges files
    nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')

    # Add 'key' column to edges
    edges['key'] = 0

    # Set indexes
    nodes = nodes.set_index(['osmid'])
    edges = edges.set_index(['u', 'v', 'key'])

    # DEM file path
    DEM_path = (f'../data/input/raster/medellin_DEM.tif')

    # Calculate elevation
    nodes, edges = src.elevation_DEM(nodes, edges, DEM_path)

    # Filter and show edges with length 0
    edges_con_longitud_cero = edges[edges['length'] == 0]
    print(f'Edges con longitud igual a 0: {len(edges_con_longitud_cero)}\n')
    if len(edges_con_longitud_cero) != 0:
        edges = edges[edges['length'] != 0]

    nodes.to_file(f'../data/output/shape/physical_variables/slope/{z}/{z}_slope_nodes.shp')
    edges.to_file(f'../data/output/shape/physical_variables/slope/{z}/{z}_slope_edges.shp')

    # Limit the study area to the neighborhood polygon
    edges = edges.reset_index()
    edges = edges.to_crs('epsg:32618')
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:32618')

    if z == 'floresta_HH':
        w = 'Alto_Alta'
    elif z == 'moravia_LH':
        w = 'Bajo_Alta'
    elif z == 'aguacatala_HL':
        w = 'Alto_Baja'

    # Load polygon files
    polygon = gpd.read_file(f'../data/input/shape/Poligonos/{w}.shp')
    polygon = polygon.to_crs('epsg:32618')
    polygon = polygon['geometry'].union_all()
    polygon = gpd.GeoDataFrame(geometry = [polygon], crs = 'epsg:32618')

    # Select the segments that are inside the polygon
    for idx, row in polygon.iterrows():
        # Filter the edges contained in the polygon
        edges_in_polygon = edges[edges['geometry'].intersects(row['geometry'])]

    # List of the nodes inside the polygon based on the 'u' and 'v' columns of the edges inside the polygon
    nodes_in_polygon = edges_in_polygon['u'].to_list() + edges_in_polygon['v'].to_list()
    # Make the nodes unique and sort it
    nodes_in_polygon = list(set(nodes_in_polygon))
    nodes_in_polygon.sort()
    # Select the nodes inside the buffer
    nodes_in_polygon = nodes.set_index('osmid').loc[nodes_in_polygon].reset_index()

    # Export
    nodes_in_polygon = nodes_in_polygon.to_crs('epsg:4326')
    edges_in_polygon = edges_in_polygon.to_crs('epsg:4326')
    nodes_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_slope_nodes.shp')
    edges_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_slope_edges.shp')


Procesando zona: floresta_HH
Edges con longitud igual a 0: 0

Procesando zona: moravia_LH
Edges con longitud igual a 0: 0

Procesando zona: aguacatala_HL
Edges con longitud igual a 0: 0



# Medellin

In [3]:
z = 'medellin'

# # Load nodes and edges files
nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')

# # Add 'key' column to edges
# edges['key'] = 0

# Set indexes
nodes = nodes.set_index(['osmid'])
edges = edges.set_index(['u', 'v', 'key'])

# DEM file path
DEM_path = (f'../data/input/raster/medellin_DEM.tif')

# Calculate elevation
nodes, edges = src.elevation_DEM(nodes, edges, DEM_path)

# Filter and show edges with length 0
edges_con_longitud_cero = edges[edges['length'] == 0]
print(f'Edges con longitud igual a 0: {len(edges_con_longitud_cero)}\n')
if len(edges_con_longitud_cero) != 0:
    edges = edges[edges['length'] != 0]

nodes.to_file(f'../data/output/shape/physical_variables/slope/{z}/{z}_slope_nodes.shp')
edges.to_file(f'../data/output/shape/physical_variables/slope/{z}/{z}_slope_edges.shp')

Edges con longitud igual a 0: 0

